In [2]:
import sys  
sys.path.insert(1, '..')
sys.path.insert(2, '../modules/')

import pickle
import os
import pandas as pd
from sunpy.time import TimeRange
import sunpy.instr.goes
import time
from time import timezone
from sunpy.net import hek
from sunpy.time import parse_time

from sunpy.net import hek
from sunpy.time import parse_time
import numpy as np
from ruffus import *
import re
import datetime
from datetime import timedelta

import dataconfig

In [3]:
WORKING_DIR = 'flare_hekQ.working'

tw = lambda x: os.path.join(WORKING_DIR, x)

# create initial time pairs for running through the hek AR query by month
time_array = []

x = pd.date_range(start='4/01/2010', end='11/30/2020', freq = 'M')

# x starts with the end date of april such that for every starting point we need to add an extra day such that we begin in the same month as the end month. 
for previous, current in zip(x, x[1:]):
    j = ([previous + timedelta(days = 1), current ])
    my_dict = {'name': '{}_{}_flare_hek'.format(previous.year, previous.month),
    			'start': j[0], 
    			'end':j[1]}
    time_array.append(my_dict)

In [11]:
time_array[0]

{'name': '2010_4_flare_hek',
 'start': Timestamp('2010-05-01 00:00:00', freq='M'),
 'end': Timestamp('2010-05-31 00:00:00', freq='M')}

In [9]:
def create_initial_files():
	for file in time_array[:1]:
		infile = None
		name = file['name']
		start = file['start']
		end = file['end']
		outfile = tw('{}.start.pickle'.format(name))
		return(infile,outfile,start,end)

In [13]:
first_call = create_initial_files()
first_call

(None,
 'flare_hekQ.working/2010_4_flare_hek.start.pickle',
 Timestamp('2010-05-01 00:00:00', freq='M'),
 Timestamp('2010-05-31 00:00:00', freq='M'))

In [74]:
def create_request(infile, outfile, start, end):


	# pickle.dump({'start': start, 'end': end, 'event_type': 'FL'}, open(outfile, 'wb'))
	

In [ ]:
# @transform(create_request, suffix('.start.pickle'), "_flares_hek_result.pickle")
def download_data(infile, outfile):
	
	start_file = pickle.load(open(infile, 'rb'))

	client = hek.HEKClient()

	event_type = start_file['event_type']

	tstart = start_file['start']

	tend = start_file['end']

	# print('started {}'.format(tstart))

	# time_range = TimeRange(t_start, t_end)

	# tstart = time_range.start
	# tend = time_range.end

	AR_results = client.search(hek.attrs.Time(tstart, tend),
	                           hek.attrs.EventType(event_type))

	pickle.dump(AR_results, open(outfile, 'wb'))

In [75]:
def parse_HEK_FL_result(infile, outfile):

	    # want to condense the results of the query into a more manageable
	    # dictionary
	    # keep event data, start time, peak time, end time, GOES-class,
	    # location, active region source (as per GOES list standard)
	    # make this into a list of dictionaries

	goes_results = pickle.load(open(infile,'rb'))

	goes_event_list = []

	for r in goes_results:
		try:
			hgc_poly_string = r['hgc_bbox']
			sep_coord_from_string_hgc = re.split('[(-)]+', hgc_poly_string)
			sep_coord_from_string_hgc = re.split('[(-)]+', hgc_poly_string)
			only_coord_string_list_hgc = sep_coord_from_string_hgc[1].split(',')
			this_coord_array_hgc = []
			for coord_pair_string_hgc in only_coord_string_list_hgc:
				numerical_coord_pairs_hgc = [np.float(coord_pair_string_hgc.split(' ')[0]), np.float(coord_pair_string_hgc.split(' ')[1])]
				this_coord_array_hgc.append(numerical_coord_pairs_hgc)
		except:
			this_coord_array_hgc = [0]

		try:
			hpc_poly_string = r['hpc_bbox']
			sep_coord_from_string_hpc = re.split('[(-)]+', hpc_poly_string)
			sep_coord_from_string_hpc = re.split('[(-)]+', hpc_poly_string)
			only_coord_string_list_hpc = sep_coord_from_string_hpc[1].split(',')
			this_coord_array_hpc = []
			for coord_pair_string_hpc in only_coord_string_list_hpc:
				numerical_coord_pairs_hpc = [np.float(coord_pair_string_hpc.split(' ')[0]), np.float(coord_pair_string_hpc.split(' ')[1])]
				this_coord_array_hpc.append(numerical_coord_pairs_hpc)
		except:
			this_coord_array_hpc = [0]


		try:
			hgs_poly_string = r['hgs_bbox']
			sep_coord_from_string_hgs = re.split('[(-)]+', hgs_poly_string)
			sep_coord_from_string_hgs = re.split('[(-)]+', hgs_poly_string)
			only_coord_string_list_hgs = sep_coord_from_string_hgs[1].split(',')
			this_coord_array_hgs = []
			for coord_pair_string_hgs in only_coord_string_list_hgs:
				numerical_coord_pairs_hgs = [np.float(coord_pair_string_hgs.split(' ')[0]), np.float(coord_pair_string_hgs.split(' ')[1])]
				this_coord_array_hgs.append(numerical_coord_pairs_hgs)
		except:
			this_coord_array_hgs = [0]
			
		if r['noposition'] == 'true':
			flare_loc_bool = 1
		else:
			flare_loc_bool = 0

		goes_event = {
			'event_date': parse_time(r['event_starttime']).strftime(
			    '%Y-%m-%d'),
			'start_time': parse_time(r['event_starttime']),
			'peak_time': parse_time(r['event_peaktime']),
			'end_time': parse_time(r['event_endtime']),
			'goes_class': str(r['fl_goescls']),
			'AR_num': r['ar_noaanum'],
			'hgs_x': r['hgs_x'],
			'hgs_y': r['hgs_y'],
			'hgs_bbox_poly': this_coord_array_hgs,
			'hgc_x': r['hgc_x'],
			'hgc_y': r['hgc_y'],
			'hgc_bbox_poly': this_coord_array_hgc,
			'hpc_x': r['hpc_x'],
			'hpc_y': r['hpc_y'],
			'hpc_bbox_poly': this_coord_array_hpc,
			'event_type': r['event_type'],
			'telescope_used': r['obs_observatory'],
			'id_institute': r['frm_institute'],
			'id_team': r['frm_identifier'],
			'search_instrument': r['search_instrument'],
			'search_channel': r['search_channelid'],
			'noposition': flare_loc_bool 
			}
		goes_event_list.append(goes_event)

	df = pd.DataFrame(goes_event_list)

	pickle.dump(df, open(outfile, 'wb'))

	# print('Ended {}'.format(tstart))


{'time_range':    <sunpy.time.timerange.TimeRange object at 0x7efcab3e0850>
     Start: 2010-01-01 00:00:00
     End:   2010-12-31 00:00:00
     Center:2010-07-02 00:00:00
     Duration:364.0 days or
            8736.0 hours or
            524160.0 minutes or
            31449600.0 seconds,
 'class_filter_start': 'C1',
 'class_filter_end': 'C9.9'}

In [76]:

@merge(parse_HEK_FL_result, 'hek_flare_db.pickle',output_dir=f'{dataconfig.DATA_DIR_PRODUCTS}')
def merge_hek_query_per_month(infiles, outfile):
	merged_df_list = []

	for infile in infiles:
		input_data = pickle.load(open(infile, 'rb'))
		merged_df_list.append(input_data)
        
	merged_df = pd.concat(merged_df_list)

	merged_df = merged_df.sort_values('start_time')

	merged_df = merged_df.reset_index(drop = True)

	pickle.dump(merged_df, open(outfile, 'wb'))

pipeline_run([merge_hek_query_per_month], multiprocess = 10, verbose = 4)

In [77]:
tstart = f'{2015}-01-01'
tend = f'{2015}-01-31'

search_result = client.search(hek.attrs.Time(tstart, tend),
	                           hek.attrs.EventType('FL'))

In [122]:
keep_columns = ['hpc_bbox',
                'hgs_bbox',
                'hgc_bbox',
                'hgc_coord',
                'event_type',
                'noposition',
                'obs_channelid',
                'hpc_y',
                'hpc_x',
                'search_instrument',
                'ar_noaanum',
                'hgs_y',
                'hgs_x',
                'event_starttime',
                'event_endtime',
                'event_peaktime',
                'fl_goescls',
                'hgs_coord',
                'obs_observatory',
                'search_observatory'
                
                ]

In [124]:
search_result.to_pandas()[keep_columns].search_observatory.value_counts()

SDO      1737
GOES      192
MAVEN      45
IRIS        4
Name: search_observatory, dtype: int64

In [89]:
for element in search_result.to_pandas().columns:
    print(element)

gs_thumburl
comment_count
hpc_bbox
frm_humanflag
hgc_coord
event_coordsys
obs_levelnum
hpc_coord
event_npixels
gs_imageurl
ar_polarity
frm_paramset
hrc_coord
event_starttime
ar_mtwilsoncls
event_type
intensmin
fl_fluence
obs_meanwavel
frm_url
skel_chaincode
bound_chaincode
noposition
fl_fluenceunit
active
intensmax
frm_versionnumber
fl_peaktempunit
fl_halphaclass
area_uncert
obs_dataprepurl
hpc_geom
hgc_bbox
intensmedian
chaincodetype
obs_channelid
event_clippedspatial
ar_noaaclass
SOL_standard
event_avg_rating
eventtype
intensunit
hpc_boundcc
event_mapurl
frm_contact
ar_penumbracls
intensmean
bound_ccstartc1
frm_name
area_atdiskcenter
frm_identifier
obs_observatory
event_description
boundbox_c2ur
obs_firstprocessingdate
boundbox_c2ll
frm_institute
hrc_bbox
refs_orig
ar_mcintoshcls
event_maskurl
bound_ccstartc2
gs_movieurl
event_score
skel_startc2
skel_startc1
fl_efoldtime
event_expires
fl_efoldtimeunit
hrc_boundcc
event_probability
intensvar
frm_daterun
event_coordunit
hpc_y
hpc_x
sea

In [81]:
# new_df = pd.DataFrame(search_result.unstack()).transpose()


AttributeError: 'HEKTable' object has no attribute 'unstack'

In [45]:

# @transform(create_request, suffix('.start.pickle'), "_goes_HEK_result.pickle")
def download_data(infile, outfile):
	start_file = pickle.load(open(infile, 'rb'))

	client = hek.HEKClient()
	event_type = 'FL'

	tstart = start_file['time_range'].start
	tend = start_file['time_range'].end

	goes_class_filter = start_file['class_filter_start']

	goes_class_filter2 = start_file['class_filter_end']



	# query the HEK for a list of events detected by the GOES instrument
	# between tstart and tend (using a GOES-class filter)
	if goes_class_filter:
	    goes_results = client.search(hek.attrs.Time(tstart, tend),
	                           hek.attrs.EventType(event_type),
	                           hek.attrs.FL.GOESCls > goes_class_filter,
	                           hek.attrs.FL.GOESCls < goes_class_filter2,
	                           hek.attrs.OBS.Observatory == 'GOES')
	else:
	    goes_results = client.search(hek.attrs.Time(tstart, tend),
	                           hek.attrs.EventType(event_type),
	                           hek.attrs.OBS.Observatory == 'GOES')

	pickle.dump(goes_results, open(outfile, 'wb'))